### Shapash Model Overview
https://shapash.readthedocs.io/en/latest/

##### Here we will:
Understand how to create a Shapash SmartPredictor to make prediction and have local explanation in production with a simple use case.

This tutorial describes the different steps from training the model to Shapash SmartPredictor deployment. A more detailed tutorial allows you to know more about the SmartPredictor Object.

Contents:

- Build a Regressor
- Compile Shapash SmartExplainer
- From Shapash SmartExplainer to SmartPredictor
- Save Shapash Smartpredictor Object in pickle file
- Make a prediction

In [1]:
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [2]:
df=sns.load_dataset('tips')

In [3]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [4]:
### Divide the dataset into independent and dependent dataset
y=df['tip']
X=df[df.columns.difference(['tip'])]

In [5]:
X.head()

,day,sex,size,smoker,time,total_bill
0,Sun,Female,2,No,Dinner,16.99
1,Sun,Male,3,No,Dinner,10.34
2,Sun,Male,3,No,Dinner,21.01
3,Sun,Male,2,No,Dinner,23.68
4,Sun,Female,4,No,Dinner,24.59


In [6]:
y

0      1.01
1      1.66
2      3.50
3      3.31
4      3.61
       ... 
239    5.92
240    2.00
241    2.00
242    1.75
243    3.00
Name: tip, Length: 244, dtype: float64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.3 KB


In [8]:
X['day']

0       Sun
1       Sun
2       Sun
3       Sun
4       Sun
       ... 
239     Sat
240     Sat
241     Sat
242     Sat
243    Thur
Name: day, Length: 244, dtype: category
Categories (4, object): [Thur, Fri, Sat, Sun]

In [9]:
X['day'].cat.codes

0      3
1      3
2      3
3      3
4      3
      ..
239    2
240    2
241    2
242    2
243    0
Length: 244, dtype: int8

In [10]:
X['day']=X['day'].cat.codes
X['sex']=X['sex'].cat.codes
X['smoker']=X['smoker'].cat.codes
X['time']=X['time'].cat.codes

In [11]:
X

,day,sex,size,smoker,time,total_bill
0,3,1,2,1,1,16.99
1,3,0,3,1,1,10.34
2,3,0,3,1,1,21.01
3,3,0,2,1,1,23.68
4,3,1,4,1,1,24.59
...,...,...,...,...,...,...
239,2,0,3,1,1,29.03
240,2,1,2,0,1,27.18
241,2,0,2,0,1,22.67
242,2,0,2,1,1,17.82


In [12]:
### Train Test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.75,random_state=1)

In [13]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators=200).fit(X_train,y_train)

#### Lets Understand Our Model With Shapash 
In this section, we use the SmartExplainer Object from shapash.

- It allows users to understand how the model works with the specified data.
- This object must be used only for data mining step. Shapash provides another object for deployment.


In [14]:
from shapash.explainer.smart_explainer import SmartExplainer

In [15]:
xpl = SmartExplainer(model=regressor)

In [16]:
xpl.compile(
    x=X_test
)

In [17]:
xpl

#### Lets Understand the results of your trained model
Then, we can easily get a first summary of the explanation of the model results.

- Here, we chose to get the 3 most contributive features for each prediction.
- We used a wording to get features names more understandable in operationnal case.

In [18]:
app = xpl.run_app(title_story='Tips Dataset')

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:root:Your Shapash application run on http://192.168.1.3:8050/


Dash is running on http://0.0.0.0:8050/



INFO:root:Use the method .kill() to down your app.
INFO:shapash.webapp.smart_app:Dash is running on http://0.0.0.0:8050/



 * Serving Flask app "shapash.webapp.smart_app" (lazy loading)
 * Environment: production


In [19]:
predictor = xpl.to_smartpredictor()

   Use a production WSGI server instead.
 * Debug mode: off


In [20]:
predictor.save('./predictor.pkl')

INFO:werkzeug: * Running on http://0.0.0.0:8050/ (Press CTRL+C to quit)


In [21]:
from shapash.utils.load_smartpredictor import load_smartpredictor
predictor_load = load_smartpredictor('./predictor.pkl')

#### Make a prediction with your SmartPredictor
In order to make new predictions and summarize local explainability of your model on new datasets, you can use the method add_input of the SmartPredictor.

- The add_input method is the first step to add a dataset for prediction and explainability.
- It checks the structure of the dataset, the prediction and the contribution if specified.
- It applies the preprocessing specified in the initialisation and reorder the features with the order used by the model. (see the documentation of this method)
- In API mode, this method can handle dictionnaries data which can be received from a GET or a POST request.
- Add data
- The x input in add_input method doesn't have to be encoded, add_input applies preprocessing.

In [22]:
predictor_load.add_input(x=X, ypred=y)

In [23]:
detailed_contributions = predictor_load.detail_contributions()


In [24]:
detailed_contributions.head()

,tip,day,sex,size,smoker,time,total_bill
0,1.01,0.061577,0.106799,-0.023806,-0.034002,-0.005614,0.190548
1,1.66,0.090254,-0.055215,-0.010680,-0.069898,-0.018349,-1.212958
2,3.50,0.088065,-0.023910,0.002713,-0.004694,0.007560,0.470718
3,3.31,0.132209,-0.023233,-0.042330,0.015416,0.004982,0.112857
4,3.61,0.060763,0.059497,-0.010469,0.002614,-0.010926,-0.320577


#### Summarize explanability of the predictions
- You can use the summarize method to summarize your local explainability
- This summary can be configured with modify_mask method so that you have explainability that meets your operational needs.

In [25]:

predictor_load.modify_mask(max_contrib=3)

In [26]:
explanation = predictor_load.summarize()

In [27]:
explanation.head()

,tip,feature_1,value_1,contribution_1,feature_2,value_2,contribution_2,feature_3,value_3,contribution_3
0,1.01,total_bill,16.99,0.190548,sex,1,0.106799,day,3,0.0615766
1,1.66,total_bill,10.34,-1.21296,day,3,0.0902535,smoker,1,-0.0698982
2,3.50,total_bill,21.01,0.470718,day,3,0.0880646,sex,0,-0.0239099
3,3.31,day,3,0.132209,total_bill,23.68,0.112857,size,2,-0.0423296
4,3.61,total_bill,24.59,-0.320577,day,3,0.060763,sex,1,0.059497


# Happy Learning